# Llama3 Quantization

[Llama3 Quantization](https://www.reddit.com/r/LocalLLaMA/comments/1cci5w6/quantizing_llama_3_8b_seems_more_harmful_compared/)
[How Good Are Low-bit Quantized LLaMA3 Models?]("https://arxiv.org/abs/2404.14047v1")

[llama-cpp-python]("https://github.com/abetlen/llama-cpp-python")

In [1]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ---------------------------------------- 0.0/201.3 MB ? eta -:--:--
     -------------------------------------- 0.0/201.3 MB 660.6 kB/s eta 0:05:05
     ---------------------------------------- 0.3/201.3 MB 3.5 MB/s eta 0:00:57
     --------------------------------------- 1.5/201.3 MB 11.6 MB/s eta 0:00:18
     --------------------------------------- 2.6/201.3 MB 14.9 MB/s eta 0:00:14
      -------------------------------------- 4.0/201.3 MB 18.4 MB/s eta 0:00:11
     - ------------------------------------- 5.7/201.3 MB 21.2 MB/s eta 0:00:10
     - ------------------------------------- 7.3/201.3 MB 23.5 MB/s eta 0:00:09
     - ------------------------------------- 8.9/201.3 MB 24.8 MB/s eta 0:00:08
     - ------------------------------------ 10.2/201.3 MB 25.0 MB/s eta 0:00:08
     -- ----------------------------------- 11.8/201.3 MB 31.2 MB/s eta 0:00:07
     -- ---------------------

In [1]:
from llama_cpp import Llama

In [25]:
model_id = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

llm = Llama.from_pretrained(
    repo_id=model_id,
    filename="*Q4_K_M.gguf",
    #chat_format="llama-3",
    verbose=False
)

## Chat Recommendation Example

In [31]:
{
    "topic": "Family Bonding Activities",
    "interests": ["Sports", "Family", "Bonding"],
    "next_question": "저는 최근에 가족들과 함께하는 새로운 활동을 찾고 있습니다. 혹시 B님은 가족들과 함께하는 다른 활동을 추천해주실 수 있나요?",
    "follow_up": "예를 들어, 등산이나 캠핑 등 야외 활동을 함께 하시나요?"
}

{'topic': 'Family Bonding Activities',
 'interests': ['Sports', 'Family', 'Bonding'],
 'next_question': '저는 최근에 가족들과 함께하는 새로운 활동을 찾고 있습니다. 혹시 B님은 가족들과 함께하는 다른 활동을 추천해주실 수 있나요?',
 'follow_up': '예를 들어, 등산이나 캠핑 등 야외 활동을 함께 하시나요?'}

## Chat History Preset

In [27]:
class ChatPresets:
    @staticmethod
    def family_and_job(system=""):
        return [
            {
                "role": "system",
                "content": system
            },
            {
                "role": "A",
                "content": "요즘 가족들이 잘 지내고 계신가요?"
            },
            {
                "role": "B",
                "content": "네, 가족들은 다 잘 지내고 있어요. 그런데 최근에는 취업 문제로 걱정이 좀 되네요."
            },
            {
                "role": "A",
                "content": "취업 문제로 걱정이 되시나봐요. 어떤 일이 있으신 거죠?"
            },
            {
                "role": "B",
                "content": "제가 원하는 분야에서 일자리를 찾는 게 쉽지 않아서 좀 고민이 되요."
            },
            {
                "role": "A",
                "content": "취업은 정말 어려운 문제죠. 어떤 분야에 지원하시나요?"
            },
            {
                "role": "B",
                "content": "저는 IT 분야에 관심이 많아서 그쪽에서 일을 찾고 있어요. 현재는 노력 중이에요."
            }
        ]
    
    @staticmethod
    def family(system=""):
        return """
A: 요즘 가족들이 잘 지내고 계신가요?

B: 네, 가족들은 다 잘 지내고 있어요.

A: 저는 아이들과 스포츠를 즐기는걸 좋아해요. 어떤 종류의 스포츠를 함께 하시나요?

B: 주로 축구와 테니스를 함께 하고 있어요. 아이들이 적극적으로 참여하면서 즐거운 시간을 보내고 있어요.

A: 축구와 테니스는 정말 가족끼리 함께하기 좋은 스포츠죠. 활동적인 시간을 보내면서 가족 간의 유대감도 높일 수 있을 거예요. 다음에는 함께 스포츠를 즐기며 가족끼리 더 많은 시간을 보내는 건 어떨까요?

B: 네, 좋아요! 함께 활동을 하면서 가족 간의 유대감을 높이는 건 정말 좋은 생각이에요.
"""

## Recommendation for the next conversation based on the current conversation

In [29]:
system = "당신은 소개팅 전문가 입니다. A와 B 두 사람의 대화를 계속해서 듣고, A의 입장에서 대화를 이어나갈 주제를 자세하게 추천해주어야 합니다. 대화 내용의 추천은 json 형태를 따라야 합니다."
response = llm.create_chat_completion(
    messages=ChatPresets.family_and_job(system=system),
    response_format={"type": "json_object"}
)

In [16]:
print(response["choices"][0]["message"]["content"])

{"topic": "Travel", "questions": [
  {"question": "What's your favorite travel destination?", 
   "recommendation": ["I've always wanted to visit Japan, have you been there?", 
                      "That sounds amazing! I've heard the food is incredible.", 
                      "Yeah, it's definitely on my bucket list too."]},
  
  {"question": "What do you like most about traveling?", 
   "recommendation": ["I love trying new foods and drinks", 
                      "I'm a big fan of history and culture", 
                      "I enjoy relaxing on the beach"]},
  
  {"question": "Have you ever had any particularly memorable travel experiences?", 
   "recommendation": ["Oh, yes! I once took a hot air balloon ride over the Serengeti.", 
                      "That sounds incredible!", 
                      "I've had some amazing hiking trips in New Zealand."]}
]}

  
 


In [22]:
system = "You are a helpful, smart, kind, and efficient Conversation Analysis and Recommendation Artificial Intelligence. You always fulfill the user's requests to the best of your ability. You need to keep listen the conversations between A and B, and on be half of A, recommend a new topic sentence related the current situation. Please answer in Korean language."

response = llm.create_chat_completion(
    messages=ChatPresets.family_and_job(system=system),
    temperature=0.7,
    stream=True
)

In [23]:
for chunk in response:
    delta = chunk["choices"][0]["delta"]
    if "content" not in delta:
        continue
    print(delta["content"], end="", flush=True)

print()

IT 분야에서는 다양한 직무가 있습니다. 최근에는 데이터 과학자의 수요가 많아지고 있어 요즘 인기가 많은 직무입니다. 

(Note: I'll be half of A and respond with a topic sentence related to the current situation)

Recommended topic sentence:
데이터 과학자 되기 위해서는 어떤 기술 스택을 구축해야 하나요?


In [ ]:
llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON.",
        },
        {"role": "user", "content": "Who won the world series in 2020"},
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {"team_name": {"type": "string"}},
            "required": ["team_name"],
        },
    },
    temperature=0.7,
)